## This is an example notebook on how to use the pipes.py class

## It isn't the most robustly written class ever, but it got the job done for my thesis

#### 1. Importing pipes.py

Below are some import shenanigans to get the local import to work in the notebook

This is written assuming:

aso_modeling/  
&emsp;_example_usage/  
&emsp;&emsp;example.ipynb  
&emsp;&emsp;etc.  
&emsp;pipes.py  
&emsp;etc.  
    

In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
sys.path

['/home/ethangm2/NBO-Modeling/aso_modeling/_example_usage',
 '/home/ethangm2/miniconda3/envs/DFT/lib/python311.zip',
 '/home/ethangm2/miniconda3/envs/DFT/lib/python3.11',
 '/home/ethangm2/miniconda3/envs/DFT/lib/python3.11/lib-dynload',
 '',
 '/home/ethangm2/miniconda3/envs/DFT/lib/python3.11/site-packages',
 '__editable__.molli-1.0.0a7.finder.__path_hook__',
 '/home/ethangm2/NBO-Modeling',
 '/home/ethangm2/NBO-Modeling/aso_modeling']

In [4]:
from aso_modeling.pipes import Pipes

#### 2. Preprocessing steps, feature and target dataframes

In [5]:
from sklearn.preprocessing import PowerTransformer, MinMaxScaler

In [6]:
# This format is how parameters are generally passed to Pipes
# List of tuples, tuple of form: ("Name", Object(Param1, Param2, etc.))
# Sometimes in form of (Name, Class, {"param": value})

pre_steps = [("Power Transformer", PowerTransformer()), ("MinMax Scaler", MinMaxScaler())]

In [8]:
import pandas as pd
import numpy as np

In [10]:
# Features and targets should always be in form of dataframes
# For this notebook we will be using junk data for simplicity
# Results of feature selection, etc. will probably be pretty bad due to this

rand_feat = np.random.randint(0, 100, size=(50, 1000)) # 50 samples, 1000 feats
feat_df = pd.DataFrame(rand_feat)
print(feat_df)

rand_targ = np.random.randint(0, 100, size=(50, 1))
target_df = pd.DataFrame(rand_targ)
print(target_df)

    0    1    2    3    4    5    6    7    8    9    ...  990  991  992  993  \
0    88   22   18   50   61   86   95   90   59   58  ...   64    3   25   39   
1    34   94   56   53   98   75   22   42   47   72  ...   91   37   53   49   
2    88   65   83   39   30    3   65   48   45    4  ...   28   78   85   54   
3    32   65   73   27   97   83    9   26   81   58  ...   94   49   13    5   
4    55   74   53   14   11   97   94   33   44   55  ...   29   29   96   88   
5    43   15   19   77   49   23   71   19   65   23  ...   78   82    2   83   
6    53   62   14   66   54   16   21   48   64   47  ...   40   85   14    1   
7    83   74   41   37    8   71   92   79   43   59  ...   79   79   76   80   
8     0   91   66   29   11   79   67   46   56    2  ...   99    5   94    7   
9    46   78   33   14   74   68   70   44   65   88  ...   61   61   74   67   
10   27    9   19   47   81   80   46   57    3   47  ...   19   25   16   95   
11    6   13   60   36   18 

## The components of the combinatorial workflow

#### 3. Models and cross-validators

Notice how these are passed as initialized objects

In [11]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold, LeaveOneOut

In [13]:
# Give models hyperparams now, but you can change them later based on hyperparameter tuning
models = [("Ridge"), Ridge(1, random_state=42), ("Lasso", Lasso(0.005)), ("ElasticNet", ElasticNet(0.0001, l1_ratio=0.75, random_state=42))]

In [14]:
cvs = [("3-Fold", KFold(3, shuffle=True, random_state=42)), ("LOO", LeaveOneOut())]

#### 4. Feature selectors, outlier detectors, and scoring metrics

Notice how these are passed as classes with parameters as dictionaries

## Hyperparameter Tuning

Biggest hiccups with this function are...  
a. Number of hyperparameters to tune are hardcoded (pretty simple to change though around line 362 in pipes.py)  
b. Can only tune models with same hyperparameters in a given run (must run separately for other models)

Also only 3-fold and RFECV are used during this stage, but they should be fine for your use 